In [85]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from functions import merge_data
from functions import load_medicare_data
from functions import load_respiratory_disease_data
from os.path import join as oj
import math
import re
import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Adding FIPS to hospital data

In [82]:
#icu['cnty_name'] = icu.apply(lambda x: re.sub('[^a-zA-Z]+', '', x['cnty_name']).lower(), axis=1)
hospitals = pd.read_csv("data/hospital_level_info/02_hospital_info.csv")
county_to_fips = dict(zip(zip(hospitals['COUNTY'], hospitals['STATE']), hospitals['COUNTYFIPS']))
hospital_level_info_merged = pd.read_csv("data/hospital_level_info/03_hospital_level_info_merged.csv")
def map_county_to_fips(name, st):
    if type(name) is str:
        #name = re.sub('[^a-zA-Z]+', '', county_name).lower()
        if (name, st) in county_to_fips and county_to_fips[(name, st)] != "NOT AVAILABLE":
            return int(county_to_fips[(name, st)])
    return np.nan
    
hospital_level_info_merged['countyFIPS'] = hospital_level_info_merged.apply(lambda x: map_county_to_fips(x['County Name_y'], x['State_x']), 
                                                                            axis=1).astype('Int64')

In [83]:
len(set(hospital_level_info_merged['countyFIPS'].values))

2402

In [95]:
hospital_level_info_merged['countyFIPS'] = hospital_level_info_merged['countyFIPS'].astype(float)

In [109]:
from load_data import load_county_level
df = load_county_level()
print(df.shape)

loading county level data...


/accounts/grad/xsli/covid-19-geographic-risk-prediction/functions/merge_data.py:40: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  diabetes = pd.read_csv(diabetes, skiprows = 2, skipfooter = 1)


ks Index(['countyFIPS', 'Respiratory Mortality'], dtype='object')
(3114, 7305)


In [110]:
county_level = df
hospital_county_merged = hospital_level_info_merged.merge(county_level, how='left', on='countyFIPS')

In [111]:
print(hospital_county_merged['#ICU_beds'].values[:10])
hospital_county_merged['countyFIPS'].values[:10]

[ 66.  20.  48.  13.   0.  nan 471.  nan  32. 471.]


array([1069., 1095., 1077., 1039., 1041.,   nan, 1073.,   nan, 1117.,
       1073.])

In [103]:
hospital_county_merged.keys()

Index(['Unnamed: 0', 'CMS Certification Number', 'Facility Name_x',
       'Street Address_x', 'City_x', 'State_x', 'ZIP', 'System Affiliation',
       'Type of Facility', 'Total Employees',
       ...
       'DiabetesPercentage', 'Respiratory Mortality', 'dem_to_rep_ratio',
       '#Hospitals', '#ICU_beds', 'HeartDiseaseMortality', 'StrokeMortality',
       'Smokers_Percentage', 'FracMale2017', '#FTEHospitalTotal2017'],
      dtype='object', length=7491)